## 5.6 再帰と反復の応用

In [442]:
# 問題** オセロ
import numpy as np

def place(p,t,x,y):
    
    for i in range(-1, 2):
        for j in range(-1, 2):
            # search
            if i==0 and j==0:
                continue
            else:
                n_turn=get_n_turn(p, t, x, y, i, j)
            #turn
            for k in range(n_turn):
                p[y+(k+1)*i][x+(k+1)*j] *= -1
            if n_turn!=0:
                p[y][x] = t
    return p

def score(p):
    return len(np.where(p==1)[0]) - len(np.where(p==-1)[0])

def get_n_turn(p, t, x, y, down, right):
    n_turn=0
    is_reversible = False
    while 0+(1-down)/2<=y+n_turn*down<4-(down+1)/2 and 0+(1-right)/2<=x+n_turn*right<4-(right+1)/2:
        if p[y+(n_turn+1)*down][x+(n_turn+1)*right] == -1*t:
            n_turn += 1
        elif p[y+(n_turn+1)*down][x+(n_turn+1)*right] == 0:
            n_turn = 0
            break
        elif p[y+(n_turn+1)*down][x+(n_turn+1)*right] == t:
            is_reversible = True
            break
    if not is_reversible:
        n_turn=0
    return n_turn

def is_puttable(p, t, x, y):
    n_turns = []
    for i in range(-1, 2):
        for j in range(-1, 2):
            if i==0 and j==0:
                continue
            elif p[y][x]!=0:
                continue
            else:
                n_turns.append(get_n_turn(p, t, x, y, i, j))
        
    if sum(n_turns)==0:
        return False
    else:
        return True

def get_candidates(p, t):
    candidates = []
    for x in range(4):
        for y in range(4):
            if is_puttable(p, t, x, y):
                candidates.append([x, y])
    return candidates

def check(p, t):
    next_t = t*-1
    is_finished = False
    # 次がt*-1のターン
    candidates = get_candidates(p, next_t)
    if len(candidates)==0:
        # -1*tをおける数
        next_t *= -1
        candidates = get_candidates(p, next_t)
    if len(candidates)==0:
        # tも-1*tも置けない
        is_finished = True 
    return next_t, candidates, is_finished
    
def get_min_max_candidate(p_, t, candidates):
    res = []
    for candidate in candidates:
        p = p_.copy()
        p = place(p, t, candidate[0], candidate[1])
        res.append([score(p), candidate])
    res.sort()
    min_candidate = res[0][1]
    max_candidate = res[-1][1]
    return min_candidate, max_candidate
    
def minimax_search(p, t, candidates):
    # 再帰的にゲームを進行
    candidate_min, candidate_max = get_min_max_candidate(p, t, candidates)
       
    if t==1: #max search 
        next_p = place(p, t, candidate_max[0], candidate_max[1])

    elif t==-1: #min search 
        next_p = place(p, t, candidate_min[0], candidate_min[1])
    
    next_t, candidates, is_finished = check(next_p, t)
    if is_finished:
        return next_p
    
    return minimax_search(next_p, next_t, candidates)
        
def othello():
    initial_p = np.array([[0,0,0,0],
                    [0,-1,1,0],
                    [0,1,-1,0],
                    [0,0,0,0]])
    initial_turn = 1
    candidates = get_candidates(initial_p, initial_turn)
    return minimax_search(initial_p, initial_turn, candidates)

In [443]:
othello()

array([[-1,  1, -1,  1],
       [ 1, -1,  1,  1],
       [-1,  1, -1,  1],
       [ 1,  1,  1, -1]])